In [2]:

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import re

import shutil
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import timezone
import os
import ssl

from selenium.webdriver.common.action_chains import ActionChains
from IPython.display import clear_output

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import random



def open_browser():
    opts = Options()
    opts.headless = False
    #user:pass@10.10.10.10:8000
    #PROXY='114.7.27.98:8080'
    #opts.add_argument('--proxy-server=%s' % PROXY)
    
    #driver = webdriver.Chrome(desired_capabilities=capa)
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts) 
           
    return driver





### Open Browser

In [3]:
try:
    driver.quit()
except:
    pass

driver=open_browser()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638


[WDM] - Driver [C:\Users\shriv\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


<h2>Functions

In [152]:


#print(job)

def fetch_job_info(job):
    job_title=None
    job_link=None
    job_company_name=None
    company_link=None
    company_rating=None
    job_location=None
    shortDescription=None
    posting_date=None
    salary_text=None


    try:
        job_title=str(job.find("h2",{"class":"jobTitle"}).find_all("span")[-1]['title']).strip()
    except Exception as e:
        #print(e)
        pass

    
    try:
        job_link="https://indeed.com"+job.parent['href']
    except:
        pass
    
    try:
        job_company_name=str(job.find("span",{"class":"companyName"}).text).strip()
     
    except:
        pass

    try:
        company_link="https://indeed.com"+str(job.find("span",{"class":"companyName"}).find("a")['href']).strip()
    except:
        pass

    try:
        company_rating=str(job.find("span",{"class":"ratingNumber"}).text).strip()
    except:
        pass

    try:
        job_location=str(job.find("div",{"class":"companyLocation"}).text).strip()
    except:
        pass

    try:
        shortDescription=str(job.find("div",{"class":"job-snippet"}).text).strip()
    except:
        pass

    try:
        posting_date=str(job.find("span",{"class":"date"}).text).strip()
    except:
        pass

    try:
        salary_text=str(job.find("svg",{"aria-label":"Salary"}).parent.text)
    except:
        pass

    job_dict={"Company_ID":company_id,"SearchCompany":company,"JobType":job_type,"Country":search_country,
    'SearchLink':indeed_template,"JobTitle":job_title,"JobLink":job_link,"JobCompany":job_company_name,
    "CompanyLink":company_link,"CompanyRating":company_rating,"JobLocation":job_location,
    "ShortDescription":shortDescription,"PostingDate":posting_date,"Salary":salary_text}
    return job_dict



def fetch_page_jobs(driver):
    soup=BeautifulSoup(driver.page_source,'html5lib')
    jobs=soup.find_all("div",{"class":"slider_container"})
    jobs_list=list()
    #print(len(jobs))
    for job in jobs:
        #print(job)
        job_dict=fetch_job_info(job)
        jobs_list.append(job_dict)
    return jobs_list





In [153]:
#Add a condition to execute of zero jobs
company="Verizon"
company_id="1235"
master_jobs_list=list()

companies=['Verizon','Microsoft','Facebook','Google']
job_types=['Internship','']
search_country="United States"
num_pages=3
for company in companies:
    for job_type in job_types:
        company_jobtype=company+" "+job_type
        company_jobtype=company_jobtype.strip()
        for page in range(0,num_pages):
            page_count=page*10
            indeed_template=f"https://www.indeed.com/jobs?q={company_jobtype}&l={country}&start={page_count}"
            print("URL:",indeed_template)
            driver.get(indeed_template)
            time.sleep(random.uniform(1,4))
            soup=BeautifulSoup(driver.page_source,'html5lib')
            jobs=fetch_page_jobs(driver)
            if(len(jobs)==0):
                print("No Jobs Found. Skipping")
                break
            master_jobs_list+=jobs
            print("Jobs Fetched Till Now:",len(master_jobs_list))
    time.sleep(random.uniform(1,4))
        

URL: https://www.indeed.com/jobs?q=Verizon Internship&l=United States&start=0
Jobs Fetched Till Now: 15
URL: https://www.indeed.com/jobs?q=Verizon Internship&l=United States&start=10
Jobs Fetched Till Now: 30
URL: https://www.indeed.com/jobs?q=Verizon Internship&l=United States&start=20
Jobs Fetched Till Now: 45
URL: https://www.indeed.com/jobs?q=Verizon&l=United States&start=0
Jobs Fetched Till Now: 60
URL: https://www.indeed.com/jobs?q=Verizon&l=United States&start=10
Jobs Fetched Till Now: 75
URL: https://www.indeed.com/jobs?q=Verizon&l=United States&start=20
Jobs Fetched Till Now: 90
URL: https://www.indeed.com/jobs?q=Microsoft Internship&l=United States&start=0
Jobs Fetched Till Now: 105
URL: https://www.indeed.com/jobs?q=Microsoft Internship&l=United States&start=10
Jobs Fetched Till Now: 120
URL: https://www.indeed.com/jobs?q=Microsoft Internship&l=United States&start=20
Jobs Fetched Till Now: 135
URL: https://www.indeed.com/jobs?q=Microsoft&l=United States&start=0
Jobs Fetched 

<h2>Fields to fetch for every job</h2>
<hr><hr>
<h3>Input</h3>
<hr><hr>
<li> Company ID
<li> Company Name
<li> Job Type
<li> search_country
<li> Search Link
<hr><hr>
<h3>Searched</h3>
<hr><hr>
<li> jobLink
<li> Title
<li> indeedCompanyName
<li> rating
<li> job location
<li> short description
<li> payPerHour
<li> PostingDate


In [155]:
df=pd.DataFrame(master_jobs_list)
df.to_csv("Jobsmk1.csv")